# **Siren MLP**

The Siren MLP, published in the paper **Implicit Neural Representations with Periodic Activation Functions** is a novel architecture that explores the usage of sinusoidal activations in fitting problems. By overfitting the model on a single sample, the Siren model can accurately reconstruct the sample and its gradients. 

In this notebook we will be training the Siren MLP on an oriented point cloud, and reconstructing its Signed Distance Function (SDF). The SDF is a function $ f(\mathbf{x} ; \mathcal{m}) : \mathbb{R}^3 \rightarrow \mathbb{R}$, i.e it maps an arbitrary point $\mathbf{x}$ of 3d space to its distance from the 3d object $\mathcal{m}$. If the point lies on the surface of $\mathcal{m}$, then the distance is 0, whereas if the point lies outside or inside the object, then the distance is positive/negative respectively.

In order to visualize and evaluate the resulting SDF we will use an isosurfacing operation called "Marching Cubes". This script will generate a mesh from the SDF, which we can then visualize as we normally would.

**Usage Instructions**

Download the .zip file from eclass and upload it on google colab. You can do that in 2 ways by following the instructions below. Extract the contents and then install all the relevant libraries (you can do so by running the cells provided below). In the "data" folder you will upload the sample you want to fit and reconstruct. The sample must be an **oriented point cloud** in ".xyz" format (that is, points along with their respective normal vectors). You can convert a mesh into an oriented point cloud using the open source tool [Meshlab](https://www.meshlab.net/).

The code cells tend to get quite large, so for the sake of simplicity we use these code cells to call scripts from .py files. You are welcome to look into the code if interested. 

**Disclaimer**

The code is taken directly from the authors' [github](https://github.com/vsitzmann/siren/tree/4df34baee3f0f9c8f351630992c1fe1f69114b5f) and it has been trimmed so that it exclusively contains SDF-relevant code. The reader is encouraged to visit their repository and play around with the various visual tools they have created.

In order to access the source files there are two options: [option 1](#option1) and [option 2](#option2):

<a name="option1"></a>

# **Option 1**

If you are connected to a google account on this computer then you can go with option 1. If you are not, option 1 will ask for your credentials. Keep in mind that it does not actually have access to your account, but it is required to download the files from the external drive.

First run the following cell and perform the authentication procedure.

In [1]:
#PyDrive to access external google drive
!pip install PyDrive

#----------Connect to google drive-----------
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


After successfully authenticating your google account you may proceed on the following code cell. Paste the drive link on the *link* string variable and run the cell. After it completes running, click refresh on the files tab to the left, and a file named "siren.zip" should appear. 

If this operation is performed successfully you may proceed with the exercise.

In [2]:
link = "https://drive.google.com/file/d/1WrtFLFhmPrFUXpzeZllEFAZ1KARZWhPp/view?usp=sharing"
start_index = link.index("/d/") + 3
end_index = link.index("/view") 

file_ID = link[start_index:end_index]

downloaded = drive.CreateFile({'id': file_ID})
downloaded.GetContentFile('siren.zip')  

<a name="option2"></a>

# **Option 2**

If for some reason you do not have a google account, are reluctant to use it, or option 1 failed for some reason, then download the file provided in the link, and after downloading upload it to this colab's workspace.

After this is complete a "siren.zip" file should appear and you may proceed with the exercise.

In [3]:
#----------Unzip the file----------
!unzip siren.zip

Archive:  siren.zip
  inflating: data/thai_statue.xyz    
  inflating: dataio.py               
  inflating: diff_operators.py       
 extracting: experiment_scripts/__init__.py  
 extracting: experiment_scripts/fwi/data_cylinder_5.mat  
  inflating: experiment_scripts/test_sdf.py  
  inflating: experiment_scripts/train_sdf.py  
  inflating: loss_functions.py       
  inflating: make_figures.py         
  inflating: meta_modules.py         
  inflating: modules.py              
  inflating: sdf_meshing.py          
  inflating: torchmeta/__init__.py   
   creating: torchmeta/datasets/
  inflating: torchmeta/datasets/__init__.py  
   creating: torchmeta/datasets/assets/
   creating: torchmeta/datasets/assets/cifar100/
   creating: torchmeta/datasets/assets/cifar100/cifar-fs/
  inflating: torchmeta/datasets/assets/cifar100/cifar-fs/test.json  
  inflating: torchmeta/datasets/assets/cifar100/cifar-fs/train.json  
  inflating: torchmeta/datasets/assets/cifar100/cifar-fs/val.json  
   creat

# **Install required modules**

In [4]:
#---Install all relevant libraries--

#Pytorch
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#Scipy, scipy image & video
!pip3 install scipy
!pip3 install scikit-image
!pip3 install scikit-video
!pip3 install plyfile
!pip3 install open3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 34.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408.6 MB 31 kB/s 
     |████████████████████████████████| 8.8 MB 7.4 MB/s 
     |████████████████████████████████| 3.1 MB 48.6 MB/s 
     |████████████████████████████████| 596 kB 63.6 MB/s 
     |████████████████████████████████| 1.1 MB 55.6 MB/s 
     |█████████████

# **Train the model by running the train script**

In order to properly run your script you are going to require access to a gpu. Click on 'Runtime' -> 'Change Runtime Type' and select 'GPU'. You can test that the code works by using the sample object "thai_statue.xyz". After verifying that you can properly train the model, replace it with the object you want and train again.

The training procedure should be relatively fast, no need for hundreads of epochs. Just 25-30 should do.

In [1]:
#Run the experiment script, make sure that you are using a cuda runtime.

!python ./experiment_scripts/train_sdf.py dolphin.xyz

Loading point cloud
Finished loading point cloud
SingleBVPNet(
  (image_downsampling): ImageDownsampling()
  (net): FCBlock(
    (net): MetaSequential(
      (0): MetaSequential(
        (0): BatchLinear(in_features=3, out_features=256, bias=True)
        (1): Sine()
      )
      (1): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (2): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (3): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (4): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=1, bias=True)
      )
    )
  )
)
Epoch 0, Total loss 123.032745, iteration time 0.208152
Epoch 50, Total loss 14.105295, iteration time 0.027626
Epoch 100, Total loss 11.327994, iteration time 0.010079
Epoch 150, Total loss 11.746126, iteration time 0.011385
Epoc

# **Assert that the model has been trained correctly**

In order to ensure that our model has been trained correctly, we will visualize the sdf using an isosurfacing operation called "marching cubes". This will create a mesh from the sdf, which should closely resemble the original one.

After running the train script for an arbitrary number of epochs, when the cell above stops executing, a directory should appear in the file explorer containing "checkpoints". Naturally, when we train a neural network we need to save its weights and biases so that we can use it in a downstream application. Checkpoints do just that. They are saved states of the same neural network, but trained for a different number of epochs.

Locate the checkpoint you want to use the test script on, copy the file name and pass it as an argument to the script below.

In [4]:
#Run the evaluation script

!python ./experiment_scripts/test_sdf.py 1350

/content
SingleBVPNet(
  (image_downsampling): ImageDownsampling()
  (net): FCBlock(
    (net): MetaSequential(
      (0): MetaSequential(
        (0): BatchLinear(in_features=3, out_features=256, bias=True)
        (1): Sine()
      )
      (1): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (2): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (3): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (4): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=1, bias=True)
      )
    )
  )
)
tcmalloc: large alloc 1073741824 bytes == 0xb3e16000 @  0x7f513ed02b6b 0x7f513ed22379 0x7f508518350e 0x7f50851757c2 0x7f50bf5302d8 0x7f50bf530de7 0x7f50bfc41928 0x7f50bf4db861 0x7f50bfb4ec8c 0x7f50c0e8bced 0x7f50c0ea5ff2 0x7f50c0f14369 0x7f50bfb7111b 0x7f513a78b542 0x59

Now download the extracted .ply file in "./logs/exp1/" and MeshLab to visualize the results. You can use any mesh visualization tool to do so. The output mesh should very closely resemble the original one. Once you verify that this is true, you can proceed to run the following code cell.

In [2]:
import os
import sys

sys.path.append("/content")

import torch
import modules, utils

import random
import math

model_type = "sine"
mode = "mlp"
resolution = 512
logging_root = "./logs"
experiment_name = "exp1"
checkpoint_to_load = "1350" #WRITE THE CHECKPOINT YOU WANT TO LOAD HERE
checkpoint_path = f"./logs/{experiment_name}/checkpoints/model_epoch_{checkpoint_to_load}.pth"

class SDFDecoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Define the model.
        if mode == 'mlp':
            self.model = modules.SingleBVPNet(type=model_type, final_layer_factor=1, in_features=3)
        elif mode == 'nerf':
            self.model = modules.SingleBVPNet(type='relu', mode='nerf', final_layer_factor=1, in_features=3)
        self.model.load_state_dict(torch.load(checkpoint_path))
        self.model.cuda()

    def forward(self, coords):
        model_in = {'coords': coords}
        return self.model(model_in)['model_out']


sdf_decoder = SDFDecoder()


SingleBVPNet(
  (image_downsampling): ImageDownsampling()
  (net): FCBlock(
    (net): MetaSequential(
      (0): MetaSequential(
        (0): BatchLinear(in_features=3, out_features=256, bias=True)
        (1): Sine()
      )
      (1): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (2): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (3): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=256, bias=True)
        (1): Sine()
      )
      (4): MetaSequential(
        (0): BatchLinear(in_features=256, out_features=1, bias=True)
      )
    )
  )
)


Once the cell above has been ran, we now have access to a trained model which we can use to get values of our object's sdf at particular points in space. We are now going to use the sdf in order to perform collision detection. The model takes as input a batch of query points ( torch.Tensor with size B x 3).

*  In your main program (c++) you will throw a ball and save its trajectory to a file, as a series of points. 
*  You will then upload the file on this colab and load the points
*  Create a torch.Tensor containing the points and feed them to the model.
*  Use the sdf's values to detect a collision in your main program, and create a response accordingly.

In [ ]:
#EXAMPLE - Creating a torch tensor of points, querying the model and printing the output.
point = torch.Tensor([[0,-0.2,0],[-1,0,1]]).cuda()
out = sdf_decoder(point)
print(out)

tensor([[0.0899],
        [0.0761]], device='cuda:0', grad_fn=<AddBackward0>)


# **Collision Detection using Neural Network**

In [ ]:
#You can begin writing your code in this cell, using the provided rough outline:
#------------------------------------------------------------------------------#
def circlePoints(centerX, centerY, centerZ, radious):
  phi = [] #0-2π
  theta = [] #0-π
  randomPoints = []
  for i in range(100):
    phi.append(random.random() * 2*math.pi )
    theta.append(random.random() * math.pi )

  for j in range(len(phi)):
    x = radious * math.cos(phi[j]) * math.sin(theta[j])
    y = radious * math.sin(phi[j]) * math.sin(theta[j])
    z = radious * math.cos(theta[j])

    randomPoints.append([centerX+x,centerY+y,centerZ+z])

  return randomPoints

# -1 -> collision
# 1  -> no collision
def isCollision(randomPoints):
  pointTensor = torch.Tensor(randomPoints).cuda()
  dist = sdf_decoder(pointTensor)
  dist = dist.tolist()
  for i in dist:
    if (i[0] < 0):
      return -1

  return 1

#------------------------------------------------------------------------------#

#Load the trajectory points from the file
points = []
with open('data/trajectory.txt', 'r') as f:
  for line in f.readlines():
    lineSTR = line.strip().split("\t")
    points.append([float(p) for p in lineSTR])
f.close()

radious = points[0][0]
points = points[1:]

"""
#Build a torch Tensor
radious = points[0]
points = points[1:]
pointTensor = torch.Tensor(points[1:]).cuda()

#Run the model
dist = sdf_decoder(pointTensor)
"""

isInside = []
for point in points:
  points_to_check = circlePoints(point[0], point[1], point[2], radious)
  result = isCollision(points_to_check)
  isInside.append(result)

#Save the output to a file
with open('data/results_SDF_SIREN.txt', 'w') as f:
    for index, value in enumerate(isInside):
      f.write(f"{points[index][0]}\t{points[index][1]}\t{points[index][2]}\t{value}" )
      if (index != len(isInside)-1 ):
        f.write(f'\n')
f.close()

#Now go back to your original program and use the provided sdf values. If they
#are <0 it means there's a collision!

# **Extract SDF using Neural Network**

In [30]:
import numpy as np
import scipy.io

# AABB limits of recovered mesh
minX = -1.0; maxX =1.0
minY = -0.5; maxY= 0.5
minZ = -0.4; maxZ =0.4

xi = np.linspace(minX, maxX, num=100)
yi = np.linspace(maxY, minY, num=50)
zi = np.linspace(minZ, maxZ, num=50)
SDF = np.zeros((len(yi), len(xi), len(zi)))

for count_x, x in enumerate(xi):
  print(count_x)
  for count_y, y in enumerate(yi):
    for count_z, z in enumerate(zi):
      point = torch.Tensor([[x,y,z]]).cuda()
      out = sdf_decoder(point)
      SDF[count_y][count_x][count_z] = out

scipy.io.savemat('data/SDF.mat', {"SDF":SDF})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


**Common Errors**



*   "Directory/File not found" -> Open the train_sdf/test_sdf scripts and play around with the checkpoint paths
*   "Large memory allocation" -> Reduce the "resolution" parameter in the test_sdf.py script.
*   "Module PIL has not attribute "Resampling"" -> expand the call stack and locate the file that causes the error. There should be several instances of using "PIL.Image.Resampling.BICUBIC". If you replace all of them with PIL.Image.BICUBIC the code should run without problems.

Make sure to use google. All the tools you'll need are available, you just need to search for them :)